In [1]:
import pandas as pd

In [4]:
df_questao_1 = pd.read_csv('questao_1.csv', index_col='num_compra')
df_questao_2 = pd.read_csv('questao_2.csv', index_col='Loja')

## Primeira Consulta: Quem mais vendeu em tal unidade

Devido a certa ambiguidade na consulta, serão utilizadas duas aborgadens.

Abordagem 1: Dada uma unidade como entrada, qual vendedor teve maior número de vendas nela.

Abordagem 2: Para cada unidade, qual vendedor teve maior número de vendas.

Ideias adicionais:

    - Top 5 vendedores por unidade
    - Top 5 vendedores por valor de venda

### Abordagem 1

In [112]:
unidade_consultada = 'Loja Manoa'

In [113]:
# Seleciono as instâncias correspondentes à unidade solicitada
df_unidade = df_questao_1[df_questao_1['Filial'] == unidade_consultada]

# Extraio o ID dos vendedores
vendedores = df_unidade['usuario'].unique()

# Crio uma lista de tuplas no formato (quantidade de vendas do vendedor, ID do vendedor) e ordeno a lista de forma decrescente com base na quantidade de vendas
rank = sorted([(df_unidade[df_unidade['usuario'] == vendedor].shape[0], vendedor) for vendedor in vendedores], key=lambda x: -x[0])

# Seleciono as colunas de Nome e Usuario que correspondem ao identificador do vendedor que é o primeiro da lista
df_unidade[df_unidade['usuario'] == rank[0][1]][['nome', 'usuario']].drop_duplicates()

,nome,usuario
num_compra,,
24790283,Alessandra,465620


### Abordagem 2

array(['Loja Armando Mendes', 'Loja Manoa', 'Loja Cachoerinha'],
      dtype=object)

In [114]:
unidades = df_questao_1['Filial'].unique()
vendedores = df_questao_1['usuario'].unique()
top_vendedores = []

for unidade in unidades:
    df_unidade = df_questao_1[df_questao_1['Filial'] == unidade]
    top_vendedores.append(sorted([(df_unidade[df_unidade['usuario'] == vendedor].shape[0], vendedor) for vendedor in vendedores], key=lambda x: -x[0])[0][1])

df_questao_1[df_questao_1['usuario'].isin(top_vendedores)][['nome', 'usuario', 'Filial']].drop_duplicates()

,nome,usuario,Filial
num_compra,,,
24790283,Alessandra,465620,Loja Manoa
25455524,Marina,466205,Loja Armando Mendes
23599621,Giselle,842680,Loja Cachoerinha


## Segunda Consulta: venda por vendedor(nome)

Total de vendas por vendedor. A consulta será feita por nome, porém assumindo que há vendedores com nomes repetidos e que possuem usuários diferentes,
será feita uma lista com o total de vendas de cada vendedor com o nome solicitado que possui um usuário diferente.


In [148]:
vendedor_procurado = 'Daniel'

In [149]:
vendedores = df_questao_1[df_questao_1['nome'] == vendedor_procurado]['usuario'].unique()
vendas_vendedor = [(df_questao_1[df_questao_1['usuario'] == vendedor].shape[0], vendedor) for vendedor in vendedores]
total_vendas = [x[0] for x in vendas_vendedor]
vendedor = [x[1] for x in vendas_vendedor]
nome = [vendedor_procurado for x in vendas_vendedor]

In [150]:
dicionario = {
    'Total de Vendas' : total_vendas,
    'Vendedor' : vendedor,
    'Nome' : nome 
}

In [151]:
df_total_vendas = pd.DataFrame(dicionario)

In [152]:
df_total_vendas

,Total de Vendas,Vendedor,Nome
0,3,295,Daniel
1,3,30646,Daniel
2,1,35553,Daniel
3,1,40859,Daniel
4,9,56424,Daniel
5,12,66657,Daniel
6,3,114728,Daniel
7,1,139130,Daniel
8,3,219436,Daniel
9,6,285290,Daniel


## Terceira Consulta: produto mais vendido